In [ ]:
# pip install numpy pandas
# pip install trl
# pip install sentence_transformers
# pip install -U bitsandbytes
# pip install langchain langchain-community pypdf pdfplumber faiss-cpu
# pip install langchain-ollama
# pip install rank_bm25

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 이건 진짜로 그냥 필요없음
# pip install -U transformers
# pip install -U accelerate
# pip install kiwipiepy

In [9]:
import os
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

from langchain_community.document_loaders import PyPDFLoader # 1.로드
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # 2.청크
from langchain.embeddings import HuggingFaceEmbeddings # 3. 임베딩 모델
from langchain.vectorstores import FAISS # 3. 벡터 저장
from langchain.retrievers import BM25Retriever, EnsembleRetriever # 4. 검색 기법
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

from transformers import pipeline

/home/minhai/.local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:
# 1. 문서 로드 (pdf페이지로 나눠짐)
loader = PyPDFLoader("/home/minhai/2024-2-DSCD-Synnovation-3/한화 개인용 자동차보험.pdf")
documents = loader.load()

In [11]:
# 2. 문서를 적절한 크기의 조각으로 청크 (split)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30) # 청크 중 중복되는 부분 크기
  #seprator 공백이면 공백 기준으로 청크를.. \n을 기본적으로 사용 (이거는 순차적으로 진행해줌)

chunks = text_splitter.split_documents(documents)
chunks = chunks[11:] #불필요한 청크 제외(목차 등)

# 각 청크에 id 부여
for i, chunk in enumerate(chunks):
    chunk.metadata['doc_id'] = i

print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in chunks)}")

생성된 텍스트 청크 수: 702
각 청크의 길이: [476, 498, 406, 499, 482, 497, 493, 102, 489, 491, 485, 336, 469, 495, 499, 124, 494, 486, 478, 491, 247, 494, 493, 492, 489, 495, 140, 499, 496, 493, 497, 462, 496, 478, 486, 490, 491, 416, 492, 475, 499, 380, 497, 474, 496, 217, 497, 497, 474, 488, 432, 490, 468, 497, 493, 383, 493, 497, 123, 159, 500, 491, 231, 489, 476, 487, 398, 458, 468, 485, 482, 495, 475, 469, 94, 488, 475, 488, 498, 232, 492, 457, 472, 498, 454, 136, 498, 491, 468, 465, 492, 479, 352, 477, 481, 476, 491, 472, 483, 186, 492, 493, 458, 470, 488, 486, 499, 464, 483, 492, 489, 487, 75, 466, 482, 498, 470, 493, 478, 158, 478, 484, 472, 498, 477, 492, 465, 499, 478, 481, 481, 481, 52, 462, 470, 483, 474, 473, 480, 210, 486, 468, 494, 493, 470, 450, 500, 463, 474, 496, 468, 456, 329, 473, 455, 483, 480, 482, 488, 153, 491, 480, 479, 463, 480, 325, 484, 470, 473, 497, 475, 493, 466, 99, 459, 469, 489, 465, 493, 450, 349, 489, 487, 450, 483, 473, 477, 494, 105, 489, 492, 486, 492, 460, 474, 

In [ ]:
# # qa 데이터셋을 위해 파일 저장 

# # meatadata에서 정보 추출
# os.path.split(chunks[0].metadata['source'])[1].split('_')[0]

# # 메타데이터가 저장된 데이터 프레임 제작
# final_docs = []
# for i, doc in enumerate(chunks):
#     new_doc = doc.copy()
#     new_doc.metadata['doc_id'] = i # metadata에 doc id를 추가
#     new_doc.metadata['source'] = str(os.path.split(new_doc.metadata['source'])[1].split('_')[0])
#     # corp_name = str(os.path.split(new_doc.metadata['source'])[1].split('_')[0]) # metadata에서 정보를 추출하여 page_content에 추가
#     # new_doc.page_content = f"{new_doc.page_content}\n\n(참고: 이 문서는 {corp_name}에 대한 정보를 담고 있습니다.)"
#     final_docs.append(new_doc)

# # 문서 저장
# import json
# with open("chunks.json", "wb") as f:
#     for doc in chunks:
#         f.write(json.dumps(dict(doc)).encode('utf-8'))
#         f.write(b'\n')

# # JSONL 파일 로드하기
# from langchain_community.document_loaders import JSONLoader

# def metadata_func(record: dict, metadata: dict) -> dict: #메타 데이터 추가
#     metadata = record.get("metadata")
#     return metadata

# json_loader = JSONLoader(
#     file_path="/home/minhai/2024-2-DSCD-Synnovation-3/chunks.json",
#     jq_schema=".",
#     content_key="page_content",
#     json_lines=True,
#     metadata_func=metadata_func,
# )

# json_docs = json_loader.load()
# print("문서의 수:", len(json_docs))

# # 데이터프레임으로 변환
# test_data = []
# for doc in json_docs:
#     test_data.append({
#         'context': str(doc.page_content),
#         'source': str(doc.metadata.get('source', '')),
#         'doc_id': str(doc.metadata.get('doc_id', ''))
#     })

# df_test = pd.DataFrame(test_data)
# print(df_test.shape)
# df_test.head()
# df_test.to_csv("/home/minhai/2024-2-DSCD-Synnovation-3/자동차보험_chunks.csv")

'한화 개인용 자동차보험.pdf'